## Codi TFG Mireia Almena

Per a realitzar l'estudi s'han desenvolupat diverses funcions per avaluar i processar cada conjunt de dades. S'ha utilitzat Google Colab.

Els models utilitzats són `Catalan BERTa-v2`, `M-MOD`, `TwHIN-BERT` i `XLM-V`

Primer que tot, s'han importat les llibreries utiltzades: 

In [ ]:
import pandas as pd
import os
from google.colab import files
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer, FillMaskPipeline
from pprint import pprint
import torch
import matplotlib.pyplot as plt
import numpy as np

Després, s'han importat els conjunts de dades. Que es troben en la carpeta de datasets. 

In [ ]:
uploaded = files.upload() 

my_dataset = pd.read_csv("dataset.csv")
anafora_dataset = pd.read_csv("anafora_dataset.csv")
dom_dataset = pd.read_csv("dom_dataset.csv")
ser_estar_dataset = pd.read_csv("ser_estar_dataset.csv")

### Qualitat lingüística
Per analitzar la qualitat lingüística dels models mitjançant Masked Language Models, s’han implementat dues funcions principals. La primera detecta si la primera predicció i les cinc primeres prediccions generades pel model per a una frase amb el marcador `<mask>` corresponen a la resposta normativa o no normativa prèviament establerta, o bé si cap d’elles coincideix amb les respostes definides: 

In [ ]:
def masked_modeling(models, dataset):
  tokenizer_hf = AutoTokenizer.from_pretrained(models)
  model = AutoModelForMaskedLM.from_pretrained(models)
  if models == 'facebook/xmod-base':
    model.set_default_language("ca_ES")
  pipeline = FillMaskPipeline(model, tokenizer_hf)

#Per treure els valors buits 
  dataset = dataset.dropna() 

#Comptador per després
#'Top 1' fa referència a la primera predicció, i 'Top 5' a les cinc primeres
  normatives_top5 = 0
  normatives_top1 = 0
  no_normatives_top5= 0
  no_normatives_top1 = 0
  altres = 0

  prediccions_columna = [] #per afegir la columna de prediccions al dataset

#Posa en una variable totes les frases amb <masked>, les respostes normatives i les no normatives; i fa la predicció per a cada frase amb <masked>

  for index, row in dataset.iterrows():
    normatiu = row['Resposta normativa']
    no_normatiu = row['Resposta no normativa freqüent']
    masked = row["Masked"]
    res_hf = pipeline(masked)
    prediccions =[r['token_str'] for r in res_hf]
    prediccions_sense_espais = [pred.strip() for pred in prediccions] #fem strip perquè no surtin espais

    if normatiu == prediccions_sense_espais[0]: #utilitzem l'índex [0] per trobar la primera predicció
      normatives_top1 +=1
    if normatiu in prediccions_sense_espais:
        normatives_top5 +=1
    if no_normatiu==prediccions_sense_espais[0]:
      no_normatives_top1 +=1
    if no_normatiu in prediccions_sense_espais:
      no_normatives_top5 +=1
    if normatiu not in prediccions_sense_espais and no_normatiu not in prediccions_sense_espais:
       altres +=1

    prediccions_columna.append(prediccions_sense_espais)


#Afegim les noves columnes
  dataset = dataset.copy() #sense això em sortien errors
  dataset["Prediccions"] = prediccions_columna
  dataset["Model"] = models


  dataset["Normatives top 1"] = None
  dataset["Normatives top 5"] = None
  dataset["No normatives top 1"] = None
  dataset["No normatives top 5"] = None
  dataset["Altres"] = None

  # Assignem només a la primera fila
  dataset.loc[dataset.index[0], "Normatives top 1"] = normatives_top1
  dataset.loc[dataset.index[0], "Normatives top 5"] = normatives_top5
  dataset.loc[dataset.index[0], "No normatives top 1"] = no_normatives_top1
  dataset.loc[dataset.index[0], "No normatives top 5"] = no_normatives_top5
  dataset.loc[dataset.index[0], "Altres"] = altres

  nom_fitxer = "resultats_qualitat.csv"

  fitxer_existeix = os.path.isfile(nom_fitxer) #mira si troba el fitxer als files
  dataset.to_csv(nom_fitxer, mode='a', index=False, header=not fitxer_existeix) #si existeix fa append del nou contingut
  files.download(nom_fitxer) #ho descarrega


Fem una llista dels models per processar-los tots alhora: 

In [ ]:
llista_models = [
    'projecte-aina/roberta-base-ca-v2',
    'facebook/xmod-base',
    'Twitter/twhin-bert-large',
    'facebook/xlm-v-base']

#Cridem la funció per tots els models 
for model in llista_models:
    masked_modeling(model, my_dataset)


La segona funció reprodueix la mateixa anàlisi, però separa els resultats segons el tipus d’estructura gramatical: 

In [ ]:
def masked_modeling_tipus(models, dataset, resum_resultats): 
  tokenizer_hf = AutoTokenizer.from_pretrained(models)
  model = AutoModelForMaskedLM.from_pretrained(models)
  if models == 'facebook/xmod-base':
    model.set_default_language("ca_ES")
  pipeline = FillMaskPipeline(model, tokenizer_hf)

  dataset = dataset.dropna() #per treure els valors buits, que si no, no puc per les prediccions
  nom_fitxer = "resultats_tipus.csv"
  fitxer_existeix = os.path.isfile(nom_fitxer)

  for tipus in dataset["Tipus d'error"].unique():
    subset = dataset[dataset["Tipus d'error"] == tipus].copy()

  #Comptador per després
    normatives_top5 = 0
    normatives_top1 = 0
    no_normatives_top5= 0
    no_normatives_top1 = 0
    altres = 0
    prediccions_columna = [] #per afegir la columna de prediccions al dataset


#Posa en una variable totes les frases amb <masked>, les respostes normatives i les no normatives; i fa la predicció per a cada frase amb <masked>

    for index, row in subset.iterrows():
      normatiu = row['Resposta normativa']
      no_normatiu = row['Resposta no normativa freqüent']
      masked = row["Masked"]
      res_hf = pipeline(masked)
      prediccions =[r['token_str'] for r in res_hf]
      prediccions_sense_espais = [pred.strip() for pred in prediccions] #fem strip perquè no surtin espais

      if normatiu == prediccions_sense_espais[0]:
        normatives_top1 +=1
      if normatiu in prediccions_sense_espais:
          normatives_top5 +=1
      if no_normatiu==prediccions_sense_espais[0]:
        no_normatives_top1 +=1
      if no_normatiu in prediccions_sense_espais:
        no_normatives_top5 +=1
      if no_normatiu not in prediccions_sense_espais and normatiu not in prediccions_sense_espais:
        altres +=1

      prediccions_columna.append(prediccions_sense_espais)

      #imprimim totes les prediccions, amb f string per imprimir variables que hem creat abans dins de la cadena

    subset.to_csv(nom_fitxer, mode='a', index=False, header=not fitxer_existeix) #si existeix fa append del nou contingut
    fitxer_existeix = True

    #Afegim resultats per visualitzar les dades de forma més clara, ja que ara tenim més valors
    resum_resultats.append({
              "Model": models,
              "Tipus d'error": tipus,
              "Normatives Top 1": normatives_top1,
              "Normatives Top 5": normatives_top5,
              "No normativess Top 1": no_normatives_top1,
              "No normativess Top 5": no_normatives_top5,
              "Altres": altres
      })


resum_resultats = [] #fem una llista buida per l'append de dintre la funció

for model in llista_models:
  masked_modeling_tipus(model, my_dataset, resum_resultats)

# Ho posem tot en un nou dataframe, per veure més fàcilment els resultats
df_resum = pd.DataFrame(resum_resultats)
df_resum.to_csv("resum_resultats.csv", index=False)

# Descarreguem els arxius
files.download("resum_resultats.csv")

## Anàfores

Per l’anàlisi de les anàfores s’ha desenvolupat una funció que, en una frase com “Va ser a la Maria a qui va alegrar la Laura quan va venir a la nostra ciutat; per tant, la `<mask>` va venir a la nostra ciutat.”, detecta si la primera predicció feta pel model coincideix amb el subjecte (Laura) o l’objecte directe (Maria). 

In [ ]:
def anafora(models, dataset):
  tokenizer_hf = AutoTokenizer.from_pretrained(models)
  model = AutoModelForMaskedLM.from_pretrained(models)
  if models == 'facebook/xmod-base':
    model.set_default_language("ca_ES")
  pipeline = FillMaskPipeline(model, tokenizer_hf)

  prediccions_columna = []

#Comptador per després
  numero_subj = 0
  numero_od = 0

  dataset = dataset.dropna()

#Per cada línia, posa en una varaible l'objecte directe, el dubjecte, i les frases amb <mask>
  for index, row in dataset.iterrows():
    od = row['Objecte directe']
    subj = row['Subjecte']
    masked = row["Masked"]

#Realitza les prediccions
    res_hf = pipeline(masked)
    prediccions =[r['token_str'] for r in res_hf]
    prediccions_sense_espais = [pred.strip() for pred in prediccions] #fem strip perquè no surtin espais
    prediccions_columna.append(prediccions_sense_espais[0])

#Afegeix al comptador quan el model prediu el subjecte i quan prediu l'objecte directes
    if subj == prediccions_sense_espais[0]:
      numero_subj +=1
    if od==prediccions_sense_espais[0]:
      numero_od +=1

  dataset = dataset.copy() #Per evitar errors

#Creem les noves columnes
  dataset["Prediccions"] = prediccions_columna
  dataset["Model"] = models
  dataset["Prediccions subjecte"] = None
  dataset["Prediccions objecte directe"] = None

  #Assignem només a la primera fila
  dataset.loc[dataset.index[0], "Prediccions subjecte"] = numero_subj
  dataset.loc[dataset.index[0], "Prediccions objecte directe"] = numero_od

  #Ho exportem
  nom_fitxer = "resultats_anafora.csv"

  fitxer_existeix = os.path.isfile(nom_fitxer) #mira si troba el fitxer als files
  dataset.to_csv(nom_fitxer, mode='a', index=False, header=not fitxer_existeix) #si existeix fa append del nou contingut
  files.download(nom_fitxer) #ho descarrega

#Cridem la funció per tots els models
for model in llista_models:
  anafora(model, anafora_dataset)

Igual que en el cas anterior, s’ha afegit una funció addicional que agrupa els resultats segons el tipus d’estructura de les oracions.


In [ ]:
#funció anafora per tipus

def anafora_tipus(models, dataset):
  tokenizer_hf = AutoTokenizer.from_pretrained(models)
  model = AutoModelForMaskedLM.from_pretrained(models)
  if models == 'facebook/xmod-base':
      model.set_default_language("ca_ES")
  pipeline = FillMaskPipeline(model=model, tokenizer=tokenizer_hf)

  dataset = dataset.dropna()
  resultats_totals = []

  for tipus in dataset["Tipus"].unique(): #per cada tipus d'estructura (valor únic en Tipus) creem un subset
    subset = dataset[dataset["Tipus"] == tipus].copy() #per evitar errors

#Comptador per després
    prediccions_columna = []
    numero_subj = 0
    numero_od = 0

#Per cada línia de cada subset, posa en una varaible l'objecte directe, el dubjecte, i les frases amb <mask>
    for index, row in subset.iterrows():
      od = row['Objecte directe']
      subj = row['Subjecte']
      masked = row["Masked"]

#Realitza les prediccions
      res_hf = pipeline(masked)
      prediccions = [r['token_str'].strip() for r in res_hf]

      prediccions_columna.append(prediccions[0])

#Les afegeix al comptador
      if subj == prediccions[0]:
          numero_subj += 1
      if od == prediccions[0]:
          numero_od += 1

#Creem les noves columnes
    subset["Prediccions"] = prediccions_columna
    subset["Model"] = models
    subset["Tipus"]= tipus
    subset["Prediccions subjecte"] = None
    subset["Prediccions objecte directe"] = None
    subset.loc[subset.index[0], "Prediccions subjecte"] = numero_subj
    subset.loc[subset.index[0], "Prediccions objecte directe"] = numero_od

#Fem append perquè surtin els resultats de cada tipus d'estructura als resultats totals
    resultats_totals.append(subset)


  resultats_final = pd.concat(resultats_totals, ignore_index=True)
  resultats_final = resultats_final.dropna(subset=["Prediccions subjecte"])

#Si existeix el fitxer també es fa append dels resultats per al model següent
  nom_fitxer = "resultats_anafora_tipus.csv"
  fitxer_existeix = os.path.isfile(nom_fitxer)
  resultats_final.to_csv(nom_fitxer, mode='a', index=False, header=not fitxer_existeix)

  files.download(nom_fitxer)

#Cridem la funció per tots els models
for model in llista_models:
  anafora_tipus(model, anafora_dataset)

## DOM 
Per a les frases amb DOM, s’ha creat una funció que calcula la probabilitat logarítmica negativa de cada oració, tant amb com sense la preposició *a*. 

In [ ]:
def probabilitat_dom(models, dataset):
  tokenizer = AutoTokenizer.from_pretrained(models)
  model = AutoModelForMaskedLM.from_pretrained(models)
  if models == 'facebook/xmod-base':
    model.set_default_language("ca_ES")

  #creem variables buides per afegir les prediccions i afegir les columnes més tard
  preds_a = []
  preds_no_a = []

  #calculem les probabilitats, primer per les frases amb a
  for sentence in dataset["Frase amb a"]:
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
      outputs = model(**inputs, labels=inputs["input_ids"])
    preds_a.append(-outputs.loss.item())

  #Després per les frases sense a
  for sentence in dataset["Frase sense a "]:
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
      outputs = model(**inputs, labels=inputs["input_ids"])
    preds_no_a.append(-outputs.loss.item())


  #noves columnes
  dataset["Prediccions a"] = preds_a
  dataset["Prediccions sense a"] = preds_no_a
  dataset["Model"] = models


  #Calculem les mitjanes

  mean_a = dataset["Prediccions a"].mean()
  mean_sense_a = dataset["Prediccions sense a"].mean()
  mean_by_type = dataset.groupby("Tipus SN")[["Prediccions a", "Prediccions sense a"]].mean() #aquí calculem la mitjana segons el tipus d'estructura
  std_a = dataset["Prediccions a"].std()
  std_sense_a = dataset["Prediccions sense a"].std()

#Buidem les columnes 
  dataset["Mitjana amb a"]= None
  dataset["Mitjana sense a"]= None
  dataset["Desviació estàndard amb a"]= None
  dataset["Desviació estàndard sense a"]= None

#Afegim només a la primera línia 
  dataset.loc[dataset.index[0], "Mitjana amb a"] = mean_a
  dataset.loc[dataset.index[0], "Mitjana sense a"] = mean_sense_a
  dataset.loc[dataset.index[0],"Desviació estàndard amb a"]= std_a
  dataset.loc[dataset.index[0],"Desviació estàndard sense a"]= std_sense_a

#Canviem el títol de les columnes
  mean_by_type = mean_by_type.rename(columns={
    "Prediccions a": "Mitjana a tipus SN",
    "Prediccions sense a": "Mitjana sense a tipus SN"
  })

  dataset = dataset.merge(mean_by_type, on="Tipus SN", how="left")
  dataset = dataset.drop(columns=[col for col in dataset.columns if "Mitjanes" in col])

  #ho exportem
  nom_fitxer = "resultats_dom.csv"

  fitxer_existeix = os.path.isfile(nom_fitxer) #mira si troba el fitxer als files
  dataset.to_csv(nom_fitxer, mode='a', index=False, header=not fitxer_existeix) #si existeix fa append del nou contingut
  files.download(nom_fitxer) #ho descarrega

#Cridem la funció per tots els models
for model in llista_models:
  probabilitat_dom(model, dom_dataset)

## Ser i estar 

Per a l’anàlisi de l’ús dels verbs ser i estar, s’ha implementat una funció anàloga que compara la probabilitat de les oracions amb el verb ser o el verb estar.


In [ ]:
def probabilitat_ser_estar(models, dataset):
  tokenizer = AutoTokenizer.from_pretrained(models)
  model = AutoModelForMaskedLM.from_pretrained(models)
  if models == 'facebook/xmod-base':
    model.set_default_language("ca_ES")

  #Creem variables buides per afegir les prediccions i afegir les columnes més tard
  preds_ser = []
  preds_estar = []

  #Calculem les probabilitats, primer per les frases amb a
  for sentence in dataset["Ser"]:
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
      outputs = model(**inputs, labels=inputs["input_ids"])
    preds_ser.append(-outputs.loss.item())

  #Després per les frases sense a
  for sentence in dataset["Estar"]:
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
      outputs = model(**inputs, labels=inputs["input_ids"])
    preds_estar.append(-outputs.loss.item())


  #Noves columnes
  dataset["Prediccions ser"] = preds_ser
  dataset["Prediccions estar"] = preds_estar
  dataset["Model"] = models


  #Calculem les mitjanes

  mean_ser = dataset["Prediccions ser"].mean()
  mean_estar = dataset["Prediccions estar"].mean()
  std_ser = dataset["Prediccions ser"].std()
  std_estar = dataset["Prediccions estar"].std()

  dataset["Mitjana ser"]= None
  dataset["Mitjana estar"]= None
  dataset["Desviació estàndard ser"]= None
  dataset["Desviació estàndard estar"]= None

  dataset.loc[dataset.index[0], "Mitjana ser"] = mean_ser
  dataset.loc[dataset.index[0], "Mitjana estar"] = mean_estar
  dataset.loc[dataset.index[0],"Desviació estàndard ser"]= std_ser
  dataset.loc[dataset.index[0],"Desviació estàndard estar"]= std_estar

  #Ho exportem
  nom_fitxer = "resultats_ser_estar.csv"

  fitxer_existeix = os.path.isfile(nom_fitxer) #mira si troba el fitxer als files
  dataset.to_csv(nom_fitxer, mode='a', index=False, header=not fitxer_existeix) #si existeix fa append del nou contingut
  files.download(nom_fitxer) #ho descarrega

#Cridem la funció per tots els models
for model in llista_models:
  probabilitat_ser_estar(model, ser_estar_dataset)